In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

DataSet Link - https://www.kaggle.com/datasets/khoongweihao/covid19-xray-dataset-train-test-sets

In [ ]:
training_dir='../input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train'
testing_dir='../input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  target_folder = target_dir+target_class
  random_image = random.sample(os.listdir(target_folder), 1)
  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");
  print(f"Image shape: {img.shape}")
  return img

In [ ]:
img = view_random_image(target_dir="../input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train/",
                        target_class="NORMAL")

In [ ]:
def load_and_prep_image(filename, img_shape=224):
  img = tf.io.read_file(filename)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.resize(img, size = [img_shape, img_shape])
  img = img/255.
  return img

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1/255.)

In [ ]:
train_data=train_datagen.flow_from_directory(training_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)
valid_data=test_datagen.flow_from_directory(testing_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

In [ ]:
import tensorflow as tf
base_model=tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10,kernel_size=3,activation="relu",input_shape=(224, 224, 3)),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2,padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
base_model.summary()

In [ ]:
base_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

base_model.fit(train_data,
               epochs=10,
               steps_per_epoch=len(train_data),
               validation_data=valid_data,
               validation_steps=len(valid_data))

In [ ]:
def pred_and_plot(model, filename, class_names):
  img = load_and_prep_image(filename)
  pred = model.predict(tf.expand_dims(img, axis=0))
  pred_class = class_names[int(tf.round(pred)[0][0])]
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
import numpy as np
import pathlib
data_dir = pathlib.Path("../input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train")
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
# ../input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg
!wget https://prod-images-static.radiopaedia.org/images/52302679/2359ab3b9b9b0bfc9874f158bda46a_jumbo.jpeg

In [ ]:
pneumonia_img='./2359ab3b9b9b0bfc9874f158bda46a_jumbo.jpeg'
pneumonia=mpimg.imread("./2359ab3b9b9b0bfc9874f158bda46a_jumbo.jpeg")
plt.imshow(pneumonia)
plt.axis(False);

In [ ]:
predict_pnewmonia_img=load_and_prep_image('./2359ab3b9b9b0bfc9874f158bda46a_jumbo.jpeg')

In [ ]:
predict_pnewmonia_img

In [ ]:
pred_and_plot(base_model,pneumonia_img,class_names)

In [ ]:
import tensorflow_hub as tfhub

resnet_url='https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/5'
IMG_SHAPE=(224,224)
feature_extraction=tfhub.KerasLayer(resnet_url,
                                    trainable=False,
                                    name='Sample_Transfer_Learning',
                                    input_shape=IMG_SHAPE+(3,))

In [ ]:
resnet_model=tf.keras.models.Sequential([
                                   feature_extraction,
                                   tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(loss='binary_crossentropy', 
                     optimizer='adam',
                     metrics=['accuracy'])
resnet_model.fit(train_data,
               epochs=10,
               steps_per_epoch=len(train_data),
               validation_data=valid_data,
               validation_steps=len(valid_data))

In [ ]:
pred_and_plot(resnet_model,pneumonia_img,class_names)